In [65]:
import numpy as np
import pandas as pd

DataSet Loading And Inspection

In [66]:
df=pd.read_csv('Groceries_dataset.csv')

In [67]:
print(df.shape)
print(df.info())

(38765, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB
None


Normalise item names

In [68]:
import re

In [69]:
def clean_item(item):
    if pd.isna(item):
        return ""
    item = str(item).lower()                 # lowercase
    item = re.sub(r'[^a-z\s]', '', item)     # keep only letters & spaces
    item = re.sub(r'\s+', ' ', item).strip() # collapse multiple spaces
    return item


In [70]:
df['cleaned'] = df['itemDescription'].apply(clean_item)
print(df[['itemDescription', 'cleaned']].head(2))

  itemDescription         cleaned
0  tropical fruit  tropical fruit
1      whole milk      whole milk


Manual mapping

In [71]:
mapping = {
    # Dairy
    "milk": "dairy",
    "whole milk": "dairy",
    "butter milk": "dairy",
    "cream": "dairy",
    "cream cheese": "dairy",
    "curd": "dairy",
    "curd cheese": "dairy",
    "processed cheese": "dairy",
    "soft cheese": "dairy",
    "spread cheese": "dairy",
    "whipped sour cream": "dairy",
    "yogurt": "dairy",
    "butter": "dairy",
    "hard cheese": "dairy",
    "domestic eggs": "dairy",
    "frozen dessert": "dairy",
    
    # Bakery
    "brown bread": "bakery",
    "white bread": "bakery",
    "pastry": "bakery",
    "cake bar": "bakery",
    "rolls/buns": "bakery",
    "zwieback": "bakery",
    "baking powder": "bakery",
    "flour": "bakery",
    
    # Beverages
    "bottled water": "beverages",
    "soda": "beverages",
    "soft drinks": "beverages",
    "fruit/vegetable juice": "beverages",
    "coffee": "beverages",
    "tea": "beverages",
    "instant coffee": "beverages",
    "liqueur": "alcohol",
    "rum": "alcohol",
    "brandy": "alcohol",
    "red/blush wine": "alcohol",
    "white wine": "alcohol",
    "sparkling wine": "alcohol",
    "beer": "alcohol",
    
    # Meat & Seafood
    "beef": "meat",
    "chicken": "meat",
    "pork": "meat",
    "fish": "seafood",
    "canned fish": "seafood",
    "ham": "meat",
    "sausage": "meat",
    "meat": "meat",
    "frozen fish": "seafood",
    "frozen chicken": "meat",
    
    # Produce
    "berries": "produce",
    "citrus fruit": "produce",
    "other vegetables": "produce",
    "onions": "produce",
    "root vegetables": "produce",
    "tropical fruit": "produce",
    "pip fruit": "produce",
    "herbs": "produce",
    "salad": "produce",
    "grapes": "produce",
    "cabbage": "produce",
    "mushrooms": "produce",
    "tomatoes": "produce",
    "pot plants": "produce",
    "flower (seeds/plants)": "produce",
    
    # Household / Cleaning
    "abrasive cleaner": "household",
    "bathroom cleaner": "household",
    "detergent": "household",
    "cleaner": "household",
    "dish cleaner": "household",
    "candles": "household",
    "kitchen towels": "household",
    "napkins": "household",
    "toilet cleaner": "household",
    "sponges": "household",
    "bags": "household",
    "cling film/bags": "household",
    "aluminum foil": "household",
    "light bulbs": "household",
    "matches": "household",
    "cat food": "household",
    "dog food": "household",
    "decalcifier": "household",
    
    # Personal Care
    "baby cosmetics": "personal care",
    "soap": "personal care",
    "cosmetics": "personal care",
    "shampoo": "personal care",
    "hygiene articles": "personal care",
    "oral hygiene": "personal care",
    "dental care": "personal care",
    "razor blades": "personal care",
    "skin care": "personal care",
    "hair spray": "personal care",
    "male cosmetics": "personal care",
    "perfume": "personal care",
    
    # Snacks & Confectionery
    "candy": "snacks",
    "chocolate": "snacks",
    "popcorn": "snacks",
    "waffles": "snacks",
    "salty snack": "snacks",
    "chips": "snacks",
    "biscuits": "snacks",
    "ice cream": "snacks",
    "chewing gum": "snacks",
    "nuts/prunes": "snacks",
    "dessert": "snacks",
    "cake": "snacks",
    
    # Canned & Packaged Foods
    "canned fruit": "canned/packaged",
    "canned vegetables": "canned/packaged",
    "canned beer": "alcohol",
    "soups": "canned/packaged",
    "cereals": "canned/packaged",
    "sugar": "canned/packaged",
    "salt": "canned/packaged",
    "jam": "canned/packaged",
    "honey": "canned/packaged",
    "cooking chocolate": "canned/packaged",
    "pudding powder": "canned/packaged",
    "spices": "canned/packaged",
    "mustard": "canned/packaged",
    "ketchup": "canned/packaged",
    "mayonnaise": "canned/packaged",
    "oil": "canned/packaged",
    "vinegar": "canned/packaged",
    "pickles": "canned/packaged",
    "sauces": "canned/packaged",
    "frozen vegetables": "frozen",
    "frozen potato products": "frozen",
    "frozen meals": "frozen",
    "condensed milk": "canned/packaged",
    
    # Frozen & Ready Meals
    "frozen dessert": "frozen",
    "frozen fish": "frozen",
    "frozen chicken": "frozen",
    "frozen meals": "frozen",
    "frozen vegetables": "frozen",
    "frozen potato products": "frozen",
    
}


In [ ]:
def map_item(item):
    if item in mapping:
        return mapping[item]
    else:
        return "other"

df['mapped'] = df['cleaned'].apply(map_item)


Group items by Member

In [73]:
baskets=df.groupby('Member_number')['mapped'].apply(lambda x:list(set(x)))
baskets.head(2)

Member_number
1000    [other, snacks, alcohol, beverages, meat, pers...
1001              [other, beverages, bakery, meat, dairy]
Name: mapped, dtype: object

In [74]:
all_items = [item for sublist in baskets for item in sublist]
unique_items = sorted(set(all_items))
print(len(unique_items))

13


In [75]:
value_counts = df['mapped'].value_counts()
common_items = value_counts[value_counts >= 10].index 
filtered_baskets = baskets.apply(lambda items: [x for x in items if x in common_items])

In [ ]:
def get_all_items():
    return sorted(df['itemDescription'].str.strip().str.lower().unique())


Transforming

In [77]:
from mlxtend.preprocessing import TransactionEncoder

In [78]:
transaction = filtered_baskets.tolist()

In [79]:
t = TransactionEncoder()

In [80]:
to_array = t.fit(transaction).transform(transaction)
encoded = pd.DataFrame(to_array, columns = t.columns_)

In [81]:
print(encoded.head(2))

   alcohol  bakery  beverages  canned/packaged  dairy  frozen  household  \
0     True    True       True            False   True   False      False   
1    False    True       True            False   True   False      False   

   meat  other  personal care  produce  seafood  snacks  
0  True   True           True    False    False    True  
1  True   True          False    False    False   False  


In [82]:
df1 = pd.DataFrame(to_array.astype(int),columns = t.columns_)

In [83]:
print(df1.head(2))

   alcohol  bakery  beverages  canned/packaged  dairy  frozen  household  \
0        1       1          1                0      1       0          0   
1        0       1          1                0      1       0          0   

   meat  other  personal care  produce  seafood  snacks  
0     1      1              1        0        0       1  
1     1      1              0        0        0       0  


Fpgrowth

In [84]:
from mlxtend.frequent_patterns import fpgrowth

In [85]:
print(encoded.columns.tolist())

['alcohol', 'bakery', 'beverages', 'canned/packaged', 'dairy', 'frozen', 'household', 'meat', 'other', 'personal care', 'produce', 'seafood', 'snacks']


In [86]:
print(encoded.shape)

(3898, 13)


In [87]:
freq = fpgrowth(encoded, min_support=0.01, use_colnames=True)

In [88]:
print(encoded.columns.tolist())

['alcohol', 'bakery', 'beverages', 'canned/packaged', 'dairy', 'frozen', 'household', 'meat', 'other', 'personal care', 'produce', 'seafood', 'snacks']


Association rules

In [89]:
from mlxtend.frequent_patterns import association_rules

In [90]:
rules = association_rules(freq,metric='lift',min_threshold=1)
rules = rules[['antecedents','consequents','support','confidence','lift']]

In [91]:
filtering = rules[(rules['confidence']>0.4) & (rules['lift']>0.8)]

In [92]:
filtering.sort_values(by='lift', ascending=False).head(2)

,antecedents,consequents,support,confidence,lift
33390,"(frozen, canned/packaged, household)","(snacks, meat, produce)",0.010005,0.438202,2.385632
33368,"(snacks, meat, canned/packaged, household)","(frozen, produce)",0.010005,0.423913,2.367354


Recommendation

In [93]:
from collections import Counter

def recommend_for_basket(user_basket, rules, top_n=5):
    user_basket = set(user_basket)
    scores = Counter()
    
    for _, row in rules.iterrows():
        antecedent = set(row['antecedents'])
        if antecedent.issubset(user_basket):
            for rec in row['consequents']:
                if rec not in user_basket:
                    scores[rec] += row['confidence'] * row['lift']
                    
    return [item for item, _ in scores.most_common(top_n)]

In [94]:
def recommend_for_basket(user_basket, rules, top_n=5):
    user_basket = set(user_basket)
    scores = Counter()
    
    for _, row in rules.iterrows():
        antecedent = set(row['antecedents'])
        if antecedent.issubset(user_basket):
            consequents = list(row['consequents'])   # convert frozenset to list
            for rec in consequents:
                if rec not in user_basket:
                    scores[rec] += row['confidence'] * row['lift']
                    
    return [item for item, _ in scores.most_common(top_n)]


Sample usage

In [95]:
user_basket = ['dairy','fruits']
recommendations = recommend_for_basket(user_basket, filtering, top_n=3)

print("Your Basket:")
for item in user_basket:
    print(f"   - {item}")

print("\nRecommended Items:")
for idx, rec in enumerate(recommendations, 1):
    print(f"   {idx}. {rec}")

Your Basket:
   - dairy
   - fruits

Recommended Items:
   1. other
   2. produce
   3. beverages


In [96]:
df1.to_csv("encoded_baskets.csv", index=False)
df.to_csv("cleaned_groceries.csv", index=False)
filtered_baskets.to_frame(name="basket").to_csv("filtered_baskets.csv")